In [ ]:
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
import os

input_path = '/mnt/e/backupfrompc/ebd_US_relSep-2025.txt'

output_dir = "/mnt/d/ebirdpyarrow"
os.makedirs(output_dir, exist_ok=True)

rows_per_shard = 5_000_000
shard_index = 0

# Read in chunks
df_iter = pd.read_csv(input_path, sep="\t", quoting=1, on_bad_lines='skip', low_memory=False, chunksize=rows_per_shard)

for df in df_iter:
    # Normalize column names
    lowercase_names = [name.lower().strip().replace(' ', '_') for name in df.columns]
    df.columns = lowercase_names
    df['scientific_name'] = df['scientific_name'].str.strip().str.lower().str.replace(' ', '_')
    # Convert to PyArrow Table
    table = pa.Table.from_pandas(df, preserve_index=False)

    # Write to partitioned dataset
    pq.write_to_dataset(
        table,
        root_path=output_dir,
        partition_cols=['scientific_name'],
        compression='snappy'
    )

    shard_index += 1
    print(f"Shard {shard_index} written.")

print("Partitioned Parquet dataset created successfully.")

Shard 1 written.


In [ ]:
import polars as pl
import os

input_path = '/mnt/e/backupfrompc/ebd_US_relSep-2025.txt'
output_dir = "/mnt/d/ebirdpolars"
os.makedirs(output_dir, exist_ok=True)

# Lazy scan
df = pl.scan_csv(
    input_path,
    separator="\t",
    quote_char='"',
    ignore_errors=True,
    low_memory=True
)

# Transformations (still lazy)
df = (
    df.rename({col: col.lower().strip().replace(' ', '_') for col in df.columns})
      .with_columns(
          pl.col("scientific_name")
          .str.strip_chars()
          .str.to_lowercase()
          .str.replace(" ", "_")
      )
)

# Streaming write to Parquet
df.lazy().sink_parquet(
    pl.PartitionByKey(output_dir, by=["scientific_name"]),
    mkdir=True
)

/tmp/ipykernel_657/2108307660.py:19: PerformanceWarning: Determining the column names of a LazyFrame requires resolving its schema, which is a potentially expensive operation. Use `LazyFrame.collect_schema().names()` to get the column names without this warning.
  df.rename({col: col.lower().strip().replace(' ', '_') for col in df.columns})


In [1]:
import duckdb, re

csv_path = '/mnt/e/backupfrompc/ebd_US_relSep-2025.txt'
output_path = '/mnt/d/ebirdduckdb'

con = duckdb.connect()

# Get column names without loading data
cols = con.execute(f"""
    SELECT * FROM read_csv_auto('{csv_path}', header=True, delim='\t') LIMIT 0
""").fetchdf().columns.tolist()

# Sanitize column names for aliases
select_expr = []
for col in cols:
    new_name = re.sub(r'[^a-z0-9]+', '_', col.strip().lower())
    quoted_col = f'"{col}"'
    if col.lower() == "scientific_name":
        select_expr.append(f"lower(replace(trim({quoted_col}), ' ', '_')) AS {new_name}")
    else:
        select_expr.append(f"{quoted_col} AS {new_name}")

# Join into a single string for SELECT
columns_sql = ", ".join(select_expr)

# Get distinct species from original column name
species_list = con.execute(f"""
    SELECT DISTINCT "SCIENTIFIC NAME"
    FROM read_csv_auto('{csv_path}', header=True, delim='\t')
""").fetchdf()["SCIENTIFIC NAME"].dropna().tolist()

# Export each species separately to avoid OOM
for sp in species_list:
    safe_sp = sp.replace(" ", "_").lower()
    sql = f"""
    COPY (
        SELECT {columns_sql}
        FROM read_csv_auto('{csv_path}', header=True, delim='\t')
        WHERE scientific_name = '{sp}'
    )
    TO '{output_path}/scientific_name={safe_sp}' (FORMAT 'parquet', COMPRESSION 'snappy');
    """
    print(f"Exporting species: {sp}")
    con.execute(sql)


Exporting species: Junco hyemalis


ConversionException: Conversion Error: CSV Error on Line: 193802268
Original Line: URN:CornellLabOfOrnithology:EBIRD:OBS506530015	2025-10-08 16:48:13.391954	33576	species	avibase-8BBB3255	Common Grackle	Quiscalus quiscula				2					United States	US	Maine	US-ME	Sagadahoc	US-ME-023				43069F7NE	Popham / Seawall ISS Area	L977151	H	43.7327784	-69.8036957	2017-05-30	17:40:00	obsr43189		S37276076	Traveling	Traveling	P22	Maine Bird Atlas|International Shorebird Survey (ISS)	1003|1039	75	3.219		1	1		0	1	0			
Error when converting column "PROJECT IDENTIFIERS". Could not convert string "1003|1039" to 'BIGINT'

Column PROJECT IDENTIFIERS is being converted as type BIGINT
This type was auto-detected from the CSV file.
Possible solutions:
* Override the type for this column manually by setting the type explicitly, e.g., types={'PROJECT IDENTIFIERS': 'VARCHAR'}
* Set the sample size to a larger value to enable the auto-detection to scan more values, e.g., sample_size=-1
* Use a COPY statement to auto